In [ ]:
def getSectionHeader(row, isColHead, colStart):
    splitString = []
    sectionH = ""
    #print("Inside function getSection", row[0], ' $&$ ', row[1])
    #for i in range(len(row)):
    i = 0
    isCapsChecked = "N"
    while i < colStart and row[i] is not None:
        splitString.append(row[i])
        sectionH = sectionH + row[i]
        i+=1

    if isColHead == 'Y':
        print("section bcos of col header ", sectionH)
        return sectionH
    
    
    for j in range(len(splitString)):
        if splitString[j].upper().split()=="PART" or splitString[j].upper().split()=="SECTION":
            print("Section bcos of Part/Section", sectionH)
            return sectionH
        print("String being checked", splitString[j])
        if isCapsChecked == "N" and splitString[j].split() != "":
            isCapsChecked = "Y"
            if splitString[j].find(".") != -1:
                print("After checking", splitString[0].split(".")[0])
                res = splitString[0].split(".")[0]
                res.replace(".", "")
                print("section being checked: ", res)
                #if res.isupper() == True or res.isdigit() == True:
                if res.isupper() ==True and res.isalpha() == True:
                    print("Section bcos of Capital letter")
                    return sectionH
    return ""      

def getColumnHeaders(row_detail, prev_Col_Header, colStart):
    print(len(row_detail)," ", colStart)
    for i in range(len(row_detail)):
        if i >= colStart:
            #print(row_detail[i])
            if row_detail[i] is not None:
                print("this row is a column bcos of ", row_detail[i].split())
                return "Y"
    return "N"

def formatStringForLIDesc(string):
    start = 0
    if string[0].isdigit() == True:
        start = string.find(" ") + 1
    res = string[start:]
    res = res.replace(".", "")
    res = res.replace("(", "")
    res = res.replace("0", "")
    #print("formated result: ", res)
    return res            


print("saving done")

In [ ]:
import openpyxl
import pandas as pd

filename = "MA(BS)1c_Form(201801).xlsx"
mdfilename = 'Test4.csv'
path     = "D:/Akhil/Work/DOTS/RegAutomation/Akhil/HK/exinput"
mode     = "N"  # N(new), M (modified)

print("After specifying File details")


In [ ]:
if mode == "N":
    wb = openpyxl.load_workbook(path+"/"+filename)
    ws = wb.worksheets[0]
    print("after opening files")

    rowidx = 0
    colidx = 0
    colStartidx = 1
    columnHeaders = []
    columnHeadersList = []
    isColHeaderPresent = 'N'
    isSectionHeader = 'N'
    sectionHeader = ""
    prevSectionHeader = ""
    eachRow = []
    eachRowColour = []
    allRows = []
    liMetadata = []
    liMetadataList = []
    lineItemId = 0
    sectionId = 0
    lineItemDesc = ""
    rowDesc = ""
    isResetColStrt = "N"
    isColContinous = "N"
    
    nvl = lambda x, y: x if x is not None else y
    for row in ws.iter_rows():
        eachRow = []
        eachRowColour = []
        for cell in row:
            eachRow.append(cell.value)
            eachRowColour.append(cell.fill.start_color.index)
         
        #Check if Row is a Column Header
        columnHeaders = []
        isColHeaderPresent = getColumnHeaders(eachRow, columnHeadersList, colStartidx)
        if isColHeaderPresent == 'Y':
            print("came here")
            if isColContinous == 'N':    ## reset ColumnHeaders if new list of Column Headers
                print("came here 2")
                columnHeadersList = [] 
                isResetColStrt = "Y"
            columnHeadersList.append(eachRow)
            isColContinous = 'Y'
        else:
            isColContinous = 'N'
        print("Is Column? ", isColHeaderPresent)
        print("Col Headers are ", len(columnHeadersList))
        #Check if Row is a separate Section
        sectionHeader = getSectionHeader(eachRow, isColHeaderPresent, colStartidx)
        #print("Section Header returned is ", sectionHeader, len(sectionHeader.split()))
        if len(sectionHeader.split()) > 0 and sectionHeader != prevSectionHeader:
            print("inside if")
            prevSectionHeader = sectionHeader
            isSectionHeader = 'Y'
            sectionId+=1
        else:
            sectionHeader = prevSectionHeader
            isSectionHeader = 'N'
        print("Is Section?", isSectionHeader)
        
        if isSectionHeader != 'Y' and isColHeaderPresent != 'Y':
            print("Identifying Line Items")
            rowDesc = ""
            for eachCell in range(len(eachRow)):
                
                print("Col ", eachCell, " is ", eachRow[eachCell], " with color ", eachRowColour[eachCell])
                if eachRow[eachCell] is not None:
                    rowDesc = rowDesc + eachRow[eachCell]
                    print("RowDesc is ", rowDesc)
                
                elif eachRow[eachCell] is None and eachRowColour[eachCell] == '00000000':
                    print("Almost there ", len(columnHeadersList))
                    columnHeaders = ""
                    for eachCol in range(len(columnHeadersList)):
                        columnHeaders = columnHeaders + " " + nvl(columnHeadersList[eachCol][eachCell], "")
                    print("ColHeader is ", columnHeaders)
                    print("ColHeader length", len(columnHeaders))
                    print("ColHeader Split length", len(columnHeaders.split()))
                    if len(columnHeaders.split()) > 0 and len(rowDesc.split()) > 0:
                        
                        lineItemId+=1
                        lineItemDesc = formatStringForLIDesc(rowDesc) + formatStringForLIDesc(columnHeaders)
                        liMetadata.append(sectionId)
                        liMetadata.append(sectionHeader)
                        liMetadata.append(lineItemId)
                        liMetadata.append(lineItemDesc)


                        print("LineItem is ", lineItemId)
                        ws.cell(row = rowidx+1, column = eachCell+1).value = lineItemId
                        #ws.cell(row = rowidx, column = eachCell).value = "{Query1:CLIV:LI=",lineItemId,",MD(RLA=Y:DD=Y:LIA=Y)"
                        liMetadataList.append(liMetadata)
                        liMetadata = []

                        if isResetColStrt == "Y":
                            isResetColStrt = "N"
                            colStartidx = eachCell

        allRows.append(eachRow)
        rowidx+=1
        isColHeaderPresent = "N"
        isSectionHeader = "N"
print("processing done") 
wb.save(path+"/op_"+filename)
df = pd.DataFrame(liMetadataList, columns = ['SectionID', 'SectionName', 'LineItemId', 'LineItemDesc'] )
df.to_csv(path+'/LI_Metadata_'+mdfilename, index = None, header = True)                    
print("all done")   





        


